### Create ML Dataset merges the final event dataset with the meteor frequency data 

For every event and non event we get the frequency files pertaining to the timeperiod of that event.

Finally, we save the final csv of 4.7 million rows in 500_000 row chunks. 



In [99]:
import pandas as pd
import os
import glob
from datetime import datetime,timedelta


In [164]:
def get_file(start_time,end_time) :
    files = glob.glob(f"E:/TDVS/*")#{str(start_time.day)}")
    date_format = "%Y-%m-%dT%H-%M-%S"

    #print(len(files),start_time,end_time)
    #csv_files_after_start = csv_file_df[(csv_file_df['date']+ timedelta(seconds=10) > start_time)]
    #print(csv_files_after_start,'after start')
    #csv_between_df = csv_file_df[(csv_file_df['date']+ timedelta(seconds=10) > start_time) &(csv_file_df['date']< end_time + timedelta(seconds=10))]
    
    date_format = "%Y-%m-%dT%H-%M-%S"
    file_return = []
    
    for file in files:
        if start_time.strftime('%Y-%m-%d') in file : 
            #print('date in',file)
            for file2 in glob.glob(file + '/*') : 
                #print('hour',start_time.strftime('%H'),file2.split('T')[-1])
                if start_time.strftime('%H') in file2.split('\\')[-1].split('-')[0] : 
                    #print('hour in',file2)
                    
                    for file3 in glob.glob(file2 + '/*') : 
                       # print('file 3',file3)
                        time_str = '-'.join(file3.split('data-')[-1].split('-')[:-1])
                        
                        dt_object = datetime.strptime(time_str,date_format)
                        #print('time is',dt_object)
                        
                        if dt_object + timedelta(seconds=10)> start_time :
                            file_return.append(file3)
                            if dt_object + timedelta(seconds=10)> end_time  :
                                break
                    break
            if len(file_return) > 0 :
                break
                        # else :
                        #     print(dt_object)
                        
                        
                        # if start_time.strftime('%M') in file3.split('data-')[-1] and 'data-' in file3 : 
                        #     print('min in',file3)
    return file_return
                            #datetime.strptime(min_date, date_format)
                #start_time.strftime('%Y-%m-%d')
            #print(glob.glob(file + '/*'))
    #path = "E:\TDVS\2024-07-30(6591)\00-00(360)\data-2024-07-30T00-00-06-110652.parquet"

In [165]:
get_file(event_df.loc[2,'start_time'],event_df.loc[2,'end_time'])

['E:/TDVS\\2024-07-27(6789)\\00-00(360)\\data-2024-07-27T00-51-59-879042.parquet']

In [184]:
def get_channel_df_round_2(file_str) :
    df_parq = pd.read_parquet(file_str)
    df_parq['Time'] = df_parq['Time'].round(2)
    return df_parq.groupby('Time').mean().reset_index()

In [185]:
df_r = get_channel_df_round_2('E:/TDVS\\2024-07-27(6789)\\00-00(360)\\data-2024-07-27T00-51-59-879042.parquet')
df_r

,Time,Channel 0,Channel 1
0,4310.00,0.021815,0.074526
1,4310.01,0.021064,0.009177
2,4310.02,0.020605,-0.009844
3,4310.03,0.020775,-0.007408
4,4310.04,0.020293,0.023452
...,...,...,...
996,4319.96,0.044476,0.036613
997,4319.97,0.044550,0.066711
998,4319.98,0.045580,0.041468
999,4319.99,0.044759,0.048526


In [193]:
event_df = pd.read_csv("C:/Users/Embed/OneDrive/Desktop/SEAL_ANTENNA/AntennasProject/ml_model/event_data.csv")

event_df['start_time'] = pd.to_datetime(event_df['start_time'])
event_df['end_time'] = pd.to_datetime(event_df['end_time'])
event_df

,original_id,event,start_time,end_time,source
0,7,meteor,2024-07-26 01:50:59,2024-07-26 01:51:01,AMS
1,8,meteor,2024-07-26 06:10:59,2024-07-26 06:11:01,AMS
2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS
3,10,meteor,2024-07-27 01:13:59,2024-07-27 01:14:01,AMS
4,11,meteor,2024-07-27 01:42:59,2024-07-27 01:43:01,AMS
...,...,...,...,...,...
1012,9,rocket,2024-08-06 06:42:00,2024-08-06 06:51:00,Rocket
1013,10,rocket,2024-08-10 12:50:00,2024-08-10 12:59:00,Rocket
1014,11,rocket,2024-08-11 01:18:00,2024-08-11 02:18:00,Rocket
1015,12,rocket,2024-08-12 02:02:00,2024-08-12 02:50:00,Rocket


In [194]:
event_df['parquet_files'] = event_df.apply(lambda x: get_file(x['start_time'],x['end_time']),axis=1)
event_df = event_df[event_df['parquet_files'].apply(len) >0]
event_df = event_df.reset_index()
event_df= event_df.rename(columns={'index':'event_id'})

event_df

,event_id,original_id,event,start_time,end_time,source,parquet_files
0,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,[E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024...
1,3,10,meteor,2024-07-27 01:13:59,2024-07-27 01:14:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
2,4,11,meteor,2024-07-27 01:42:59,2024-07-27 01:43:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
3,5,12,meteor,2024-07-27 01:59:59,2024-07-27 02:00:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
4,6,13,meteor,2024-07-27 05:15:59,2024-07-27 05:16:01,AMS,[E:/TDVS\2024-07-27(6789)\05-00(360)\data-2024...
...,...,...,...,...,...,...,...
956,1012,9,rocket,2024-08-06 06:42:00,2024-08-06 06:51:00,Rocket,[E:/TDVS\2024-08-06(3887)\06-00(360)\data-2024...
957,1013,10,rocket,2024-08-10 12:50:00,2024-08-10 12:59:00,Rocket,[E:/TDVS\2024-08-10(4156)\12-00(360)\data-2024...
958,1014,11,rocket,2024-08-11 01:18:00,2024-08-11 02:18:00,Rocket,[E:/TDVS\2024-08-11(3065)\01-00(360)\data-2024...
959,1015,12,rocket,2024-08-12 02:02:00,2024-08-12 02:50:00,Rocket,[E:/TDVS\2024-08-12\02-00(360)\data-2024-08-12...


In [213]:
event_df[event_df['event_id'] == 1012]

,event_id,original_id,event,start_time,end_time,source,parquet_files
956,1012,9,rocket,2024-08-06 06:42:00,2024-08-06 06:51:00,Rocket,[E:/TDVS\2024-08-06(3887)\06-00(360)\data-2024...


In [212]:
event_df[event_df['event_id'] == 1012]['parquet_files'].values

array([list(['E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-41-57-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-07-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-17-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-27-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-37-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-47-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-42-57-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-43-07-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-43-17-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-43-27-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-43-37-277445.parquet', 'E:/TDVS\\2024-08-06(3887)\\06-00(360)\\data-2024-08-06T06-43-47-277445.parquet', 'E:

In [168]:
event_df[event_df['parquet_files'].apply(len) >0]

,event_id,original_id,event,start_time,end_time,source,parquet_files
2,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,[E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024...
3,3,10,meteor,2024-07-27 01:13:59,2024-07-27 01:14:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
4,4,11,meteor,2024-07-27 01:42:59,2024-07-27 01:43:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
5,5,12,meteor,2024-07-27 01:59:59,2024-07-27 02:00:01,AMS,[E:/TDVS\2024-07-27(6789)\01-00(360)\data-2024...
6,6,13,meteor,2024-07-27 05:15:59,2024-07-27 05:16:01,AMS,[E:/TDVS\2024-07-27(6789)\05-00(360)\data-2024...
...,...,...,...,...,...,...,...
1012,1012,9,rocket,2024-08-06 06:42:00,2024-08-06 06:51:00,Rocket,[E:/TDVS\2024-08-06(3887)\06-00(360)\data-2024...
1013,1013,10,rocket,2024-08-10 12:50:00,2024-08-10 12:59:00,Rocket,[E:/TDVS\2024-08-10(4156)\12-00(360)\data-2024...
1014,1014,11,rocket,2024-08-11 01:18:00,2024-08-11 02:18:00,Rocket,[E:/TDVS\2024-08-11(3065)\01-00(360)\data-2024...
1015,1015,12,rocket,2024-08-12 02:02:00,2024-08-12 02:50:00,Rocket,[E:/TDVS\2024-08-12\02-00(360)\data-2024-08-12...


In [208]:
parq_event_files = list(set(sum(event_df['parquet_files'],[])))
parq_event_files

['E:/TDVS\\2024-08-13(5703)\\02-00(360)\\data-2024-08-13T02-00-15-266698.parquet',
 'E:/TDVS\\2024-08-04(7249)\\07-00(360)\\data-2024-08-04T07-31-49-146162.parquet',
 'E:/TDVS\\2024-08-12\\07-00(360)\\data-2024-08-12T07-14-32-871149.parquet',
 'E:/TDVS\\2024-08-12\\02-00(360)\\data-2024-08-12T02-16-52-871149.parquet',
 'E:/TDVS\\2024-08-13(5703)\\08-00(360)\\data-2024-08-13T08-11-25-266698.parquet',
 'E:/TDVS\\2024-08-10(4156)\\12-00(360)\\data-2024-08-10T12-51-44-296679.parquet',
 'E:/TDVS\\2024-08-11(3065)\\02-00(360)\\data-2024-08-11T02-00-52-871149.parquet',
 'E:/TDVS\\2024-08-02(4228)\\01-00(360)\\data-2024-08-02T01-12-55-927603.parquet',
 'E:/TDVS\\2024-08-12\\09-00(360)\\data-2024-08-12T09-05-42-871149.parquet',
 'E:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-37-12-871149.parquet',
 'E:/TDVS\\2024-08-04(7249)\\03-00(360)\\data-2024-08-04T03-09-59-146162.parquet',
 'E:/TDVS\\2024-07-30(4824)\\10-00(360)\\data-2024-07-30T10-47-36-110652.parquet',
 'E:/TDVS\\2024-08-12\\06-00

In [209]:
len(parq_event_files)

2368

In [230]:
files = glob.glob(f"E:/TDVS/*")#{str(start_time.day)}")

non_event_files = []
print(len(files))

for x in range(1000) :
    if len(non_event_files) > 2368 :
            break
    
    for file in files[x:x+1]:
        print('glob1',len(files),x,len(non_event_files))
        print('currnet',x * (2368//len(files)),len(non_event_files))
        if len(non_event_files) > x * (2368//len(files)) :
            break
        files2 = glob.glob(file + '/*')
        for file2 in files2 :
            if len(non_event_files) > x * (2368//len(files)) :
                break 
           # print('glob2',len(glob.glob(file + '/*')),x,len(non_event_files))
            
                    
            for file3 in glob.glob(file2 + '/*') : 
              #  print('glob3',len(glob.glob(file2 + '/*')),x,len(non_event_files))
                        
                if file3 not in parq_event_files : 
                    non_event_files.append(file3)
                        
non_event_files

54
glob1 54 0 0
currnet 0 0
glob1 54 1 120
currnet 43 120
glob1 54 2 120
currnet 86 120
glob1 54 3 120
currnet 129 120
glob1 54 4 269
currnet 172 269
glob1 54 5 269
currnet 215 269
glob1 54 6 269
currnet 258 269
glob1 54 7 269
currnet 301 269
glob1 54 8 629
currnet 344 629
glob1 54 9 629
currnet 387 629
glob1 54 10 629
currnet 430 629
glob1 54 11 629
currnet 473 629
glob1 54 12 629
currnet 516 629
glob1 54 13 629
currnet 559 629
glob1 54 14 629
currnet 602 629
glob1 54 15 629
currnet 645 629
glob1 54 16 782
currnet 688 782
glob1 54 17 782
currnet 731 782
glob1 54 18 782
currnet 774 782
glob1 54 19 782
currnet 817 782
glob1 54 20 884
currnet 860 884
glob1 54 21 884
currnet 903 884
glob1 54 22 986
currnet 946 986
glob1 54 23 986
currnet 989 986
glob1 54 24 1213
currnet 1032 1213
glob1 54 25 1213
currnet 1075 1213
glob1 54 26 1213
currnet 1118 1213
glob1 54 27 1213
currnet 1161 1213
glob1 54 28 1213
currnet 1204 1213
glob1 54 29 1213
currnet 1247 1213
glob1 54 30 1573
currnet 1290 1573
gl

['E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-09-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-19-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-29-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-39-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-49-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-59-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-09-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-19-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-29-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-39-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-49-879042.parquet',
 'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-41-59-879042.parquet',
 'E:/TDVS\\2024-

In [186]:
event_df.columns

Index(['event_id', 'original_id', 'event', 'start_time', 'end_time', 'source',
       'parquet_files'],
      dtype='object')

In [249]:
df_all = pd.DataFrame()
for ind,x in event_df.iterrows() :
    print(ind)
    
    
        
    df_single_event = pd.DataFrame()
    for y in x['parquet_files'] : 
        df = get_channel_df_round_2(y)
        
        df.loc[:,'event_id'] = x['event_id']
        df.loc[:,'original_id'] = x['original_id']
        df.loc[:,'event'] = x['event']
        df.loc[:,'start_time'] = x['start_time']
        df.loc[:,'end_time'] = x['end_time']
        df.loc[:,'source'] = x['source']
        
        df.loc[:,'parquet_files'] = y 
        df_single_event = pd.concat([df_single_event,df])

        
    df_all = pd.concat([df_all,df_single_event])
df_all

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,Time,Channel 0,Channel 1,event_id,original_id,event,start_time,end_time,source,parquet_files
0,4310.00,0.021815,0.074526,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
1,4310.01,0.021064,0.009177,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
2,4310.02,0.020605,-0.009844,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
3,4310.03,0.020775,-0.007408,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
4,4310.04,0.020293,0.023452,2,9,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
...,...,...,...,...,...,...,...,...,...,...
996,122879.96,-8.515895,-8.522952,1016,13,rocket,2024-08-12 10:37:00,2024-08-12 10:46:00,Rocket,E:/TDVS\2024-08-12\10-00(360)\data-2024-08-12T...
997,122879.97,-8.509873,-8.522957,1016,13,rocket,2024-08-12 10:37:00,2024-08-12 10:46:00,Rocket,E:/TDVS\2024-08-12\10-00(360)\data-2024-08-12T...
998,122879.98,-8.516345,-8.522888,1016,13,rocket,2024-08-12 10:37:00,2024-08-12 10:46:00,Rocket,E:/TDVS\2024-08-12\10-00(360)\data-2024-08-12T...
999,122879.99,-8.514479,-8.522875,1016,13,rocket,2024-08-12 10:37:00,2024-08-12 10:46:00,Rocket,E:/TDVS\2024-08-12\10-00(360)\data-2024-08-12T...


In [245]:
df_all.iloc[5505]['parquet_files']

'E:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-37-42-871149.parquet'

In [237]:
df_all[df_all['event_id']==1016].iloc[0]['parquet_files']

'E:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-871149.parquetE:/TDVS\\2024-08-12\\10-00(360)\\data-2024-08-12T10-36-52-87114

In [250]:
df_non_event = pd.DataFrame()
non_event_id = 2000

for non_event_file in non_event_files :
    
        
    df = get_channel_df_round_2(non_event_file)
    
    df.loc[:,'event_id'] = non_event_id
    df.loc[:,'event'] = 'non_event'
    # df.loc[:,'original_id'] = x['original_id']
    # df.loc[:,'event'] = x['event']
    # df.loc[:,'start_time'] = x['start_time']
    # df.loc[:,'end_time'] = x['end_time']
    # df.loc[:,'source'] = x['source']
    
    df.loc[:,'parquet_files'] = non_event_file
    non_event_id +=1
    

    
    df_non_event = pd.concat([df_non_event,df])
df_non_event



,Time,Channel 0,Channel 1,event_id,event,parquet_files
0,0.00,-1.830519,-7.153347,2000,non_event,E:/TDVS\2024-07-26(120)\23-00(120)\data-2024-0...
1,0.01,-1.936912,-6.880141,2000,non_event,E:/TDVS\2024-07-26(120)\23-00(120)\data-2024-0...
2,0.02,-2.065095,-6.657280,2000,non_event,E:/TDVS\2024-07-26(120)\23-00(120)\data-2024-0...
3,0.03,-2.182451,-6.424141,2000,non_event,E:/TDVS\2024-07-26(120)\23-00(120)\data-2024-0...
4,0.04,-2.289692,-6.209804,2000,non_event,E:/TDVS\2024-07-26(120)\23-00(120)\data-2024-0...
...,...,...,...,...,...,...
996,2009.96,-2.441657,-2.450518,4338,non_event,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
997,2009.97,-2.441662,-2.450510,4338,non_event,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
998,2009.98,-2.441656,-2.450504,4338,non_event,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
999,2009.99,-2.441666,-2.450514,4338,non_event,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...


In [253]:
df_all_events = pd.concat([df_all,df_non_event])
df_all_events

,Time,Channel 0,Channel 1,event_id,original_id,event,start_time,end_time,source,parquet_files
0,4310.00,0.021815,0.074526,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
1,4310.01,0.021064,0.009177,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
2,4310.02,0.020605,-0.009844,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
3,4310.03,0.020775,-0.007408,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
4,4310.04,0.020293,0.023452,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
...,...,...,...,...,...,...,...,...,...,...
996,2009.96,-2.441657,-2.450518,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
997,2009.97,-2.441662,-2.450510,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
998,2009.98,-2.441656,-2.450504,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
999,2009.99,-2.441666,-2.450514,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...


596345

In [258]:
df_all_events = df_all_events.reset_index(drop=True)
df_all_events = df_all_events.drop(columns=['index'])
df_all_events

,Time,Channel 0,Channel 1,event_id,original_id,event,start_time,end_time,source,parquet_files
0,4310.00,0.021815,0.074526,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
1,4310.01,0.021064,0.009177,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
2,4310.02,0.020605,-0.009844,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
3,4310.03,0.020775,-0.007408,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
4,4310.04,0.020293,0.023452,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
...,...,...,...,...,...,...,...,...,...,...
4770761,2009.96,-2.441657,-2.450518,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770762,2009.97,-2.441662,-2.450510,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770763,2009.98,-2.441656,-2.450504,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770764,2009.99,-2.441666,-2.450514,4338,NaN,non_event,NaT,NaT,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...


In [264]:
for x in range(10) : 
    df_chunk_x = df_all_events.iloc[x * 500_000:min((x+1) * 500_000,len(df_all_events))]
    
    df_chunk_x.to_csv(f'event_frequency_dataset_{x}.csv')

In [267]:
df_ml_dataset = pd.DataFrame()
for x in range(10) : 
    df_chunk = pd.read_csv(f'event_frequency_dataset_{x}.csv',index_col=0)
    df_ml_dataset = pd.concat([df_ml_dataset,df_chunk])
df_ml_dataset

C:\Users\Embed\AppData\Local\Temp\ipykernel_34128\2490225631.py:3: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk = pd.read_csv(f'event_frequency_dataset_{x}.csv',index_col=0)


,Time,Channel 0,Channel 1,event_id,original_id,event,start_time,end_time,source,parquet_files
0,4310.00,0.021815,0.074526,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
1,4310.01,0.021064,0.009177,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
2,4310.02,0.020605,-0.009844,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
3,4310.03,0.020775,-0.007408,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
4,4310.04,0.020293,0.023452,2,9.0,meteor,2024-07-27 00:51:59,2024-07-27 00:52:01,AMS,E:/TDVS\2024-07-27(6789)\00-00(360)\data-2024-...
...,...,...,...,...,...,...,...,...,...,...
4770761,2009.96,-2.441657,-2.450518,4338,NaN,non_event,NaN,NaN,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770762,2009.97,-2.441662,-2.450510,4338,NaN,non_event,NaN,NaN,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770763,2009.98,-2.441656,-2.450504,4338,NaN,non_event,NaN,NaN,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...
4770764,2009.99,-2.441666,-2.450514,4338,NaN,non_event,NaN,NaN,NaN,E:/TDVS\2024-08-19(8628)\00-00(348)\data-2024-...


In [281]:
df_ml_dataset.groupby(['event'])['event_id'].nunique()

event
meteor        947
non_event    2339
rocket         14
Name: event_id, dtype: int64

In [255]:
df_all_events.to_csv('event_frequency_dataset.csv')

In [254]:
df_all_events.groupby('event').count()

,Time,Channel 0,Channel 1,event_id,original_id,start_time,end_time,source,parquet_files
event,,,,,,,,,
meteor,1105104,1105104,1105104,1105104,1105104,1105104,1105104,1105104,1105104
non_event,2341339,2341339,2341339,2341339,0,0,0,0,2341339
rocket,1324323,1324323,1324323,1324323,1324323,1324323,1324323,1324323,1324323


In [233]:
df_non_event.iloc[0]['parquet_files']

'E:/TDVS\\2024-07-26(120)\\23-00(120)\\data-2024-07-26T23-40-09-879042.parquet'

In [216]:
df_all.groupby('event').count()

,Time,Channel 0,Channel 1,event_id,original_id,start_time,end_time,source,parquet_files
event,,,,,,,,,
meteor,1105104,1105104,1105104,1105104,1105104,1105104,1105104,1105104,1105104
rocket,1324323,1324323,1324323,1324323,1324323,1324323,1324323,1324323,1324323
